In [172]:
excel_path = "DataGeneratorTemplatebbb.xlsx" # Path of the excel file


In [173]:
import random
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore') # ignore warnings
class SyntheticDataGenerator:
    def __init__(self):
        # Initialize necessary components and data structures
        self.tables = {}  # Dictionary to store tables and their data
        self.tokens = {}  # Dictionary to store tokens and their data
        self.today_date = datetime.now().date()  # Today's date
        self.counter =   {}# Counter for sequential values

    def add_table(self, table_name, data):
        self.tables[table_name] = data
        self.counter[table_name] = 1 # initializes the sequential counter

    def generate_sequential(self,table_name):
        current_value = self.counter[table_name]
        self.counter[table_name] += 1
        return current_value

    def random_lookup(self, referenced_tab):
        column_name = referenced_tab
        if referenced_tab not in self.tables:
            raise ValueError(f"Table '{referenced_tab}' not found.")
        if column_name not in self.tables[referenced_tab].columns.to_list():
            raise ValueError(f"Column '{column_name}' not found in table '{referenced_tab}'.")
        column_index = self.tables[referenced_tab].columns.to_list().index(column_name) # column index in the lookup table

        values = [row[column_index] for i, row in self.tables[referenced_tab].iterrows()] # extract all values
        

        return random.choice(values)
    def random_integer(self, min_value, max_value, distribution_tab=None):
        min_value = int(min_value)
        max_value = int(max_value)
        # Check if a distribution tab is provided
        if distribution_tab:
            # Check if the distribution tab exists
            if distribution_tab not in self.tables:
                raise ValueError(f"Distribution tab '{distribution_tab}' not found.")

            # Get the index of the Count and Dist columns
            count_index = 0
            dist_index = 1

            # Get the Count and Dist columns
            counts = [int(row[count_index]) for _, row in self.tables[distribution_tab].iterrows()]
            dists = [float(row[dist_index]) for _, row in self.tables[distribution_tab].iterrows()]


            # Normalize the distribution percentages
            total_dist = sum(dists)
            normalized_dists = [dist / total_dist for dist in dists]

            # Generate a random value based on the distribution
            random_value = random.choices(counts, weights=normalized_dists)[0]
        else:
            # Generate a truly random value within the specified range
            random_value = random.randint(min_value, max_value)

        return random_value
    def lookup_with_integer(self,table_name,j, integer_value, referenced_tab,gen = 1):
        '''
        j = id of the current row
    
        '''
        # Check if the referenced table exists
        if referenced_tab not in self.tables:
            raise ValueError(f"Table '{referenced_tab}' not found.")
        params = [integer_value]
        # resolve the nested commands and generate current count
        for param in params:
            if param == "Sequential":
                l =self.generate_data("Sequential")
            elif "(" in param :
                param = param.split("(")
                command_name = param[0]
                params = [p.rstrip(")") for p in param]
                aras = params[1].split(",")
                l=self.generate_data(command_name,parameters = aras)
            elif param == "GenerateCurrentCount":
                t = pd.read_csv(f"{table_name}.csv")
                if len( list(t.iterrows())) >0:
                    
                    
                    for i , row in t.iterrows():
                        if int(row.tolist()[0]) == int(j): # increment the counter for the number of generated lines of the same id : j
                            gen+=1
                        
                if gen <= 36:
                    l=gen
                elif gen % 36 !=0: 
                    l = (gen%36)
                else : 
                    l = 36 
            else:
                l = param
  
        integer_value = l
        # Get the index of the sequential integers and the corresponding values
        index_column = 0
        value_column = 1

        # Find the row where the sequential integer matches the provided value

        for _,row in self.tables[referenced_tab].iterrows():
            
            if int(row[index_column]) == integer_value:
                return row[value_column]

        # If the provided integer value is not found, raise an error
        raise ValueError(f"Integer value '{integer_value}' not found in table '{referenced_tab}'.")
    def resolve_date_reference(self, reference):
        if reference == "TODAY":
            return self.today_date.strftime("%Y%m%d")
        return reference
    def random_date(self,ll,table_name,j,cc, start_date, end_date):
        '''
        ll : current row
        j : same as before
        cc : command column number
        '''
        l = []
        # resolve the nested commands
        for param in [start_date,end_date]:
            if param == "Sequential":
                l.append(self.generate_data("Sequential"))
            elif "(" in param :
                param = param.split("(")
                command_name = param[0]
                params = [p.rstrip(")") for p in param]
                aras = params[1].split(",")
                l.append(self.generate_data(command_name,ll=ll,table_name=table_name,c = cc,parameters = aras,j=j))
            else:
                l.append(param)
        start_date,end_date = l[0],l[1]
        # Resolve references to other columns or the 'TODAY' keyword
        start_date = self.resolve_date_reference(start_date)
        end_date = self.resolve_date_reference(end_date)

        # Convert the date parameters to datetime objects
        start_date = datetime.strptime(str(start_date), "%Y%m%d").date()
        end_date = datetime.strptime(str(end_date), "%Y%m%d").date()

        # Generate a random date within the specified range
        if end_date > start_date :
            random_days = random.randint(0, (end_date - start_date).days)
            random_date = start_date + timedelta(days=random_days)
        else : 
            random_days = random.randint(0, (start_date - end_date).days)
            random_date = end_date + timedelta(days=random_days)

        return random_date.strftime("%Y%m%d")
    def random_intersection(self,ll,c,referenced_tab, *pairs):
    
        # Validate the number of pairs
        if len(pairs) % 2 != 0:
            raise ValueError("Pairs must be provided in sets of two.")

        # Initialize the intersection list with the values from the first pair
        first_pair_tab = pairs[0]
        first_pair_column = pairs[1]
        intersection_list = []

        # Iterate over the remaining pairs and perform intersections
        for i in range(0, len(pairs), 2):
            current_tab = pairs[i]
            current_column = pairs[i + 1]
            j = self.tables[referenced_tab].iloc[0].to_list().index(current_column.strip())
            col = ll[j]
            intersection_list.append(self.tables[current_tab][col].to_list())

        
            
        # If the intersection list is not empty, return a random selection
        if intersection_list:
            intersection_list[0] =  [x for x in intersection_list[0] if  isinstance(x,str)]
            intersection_list[1] =  [x for x in intersection_list[1] if isinstance(x,str)]

            intersection = set(intersection_list[0]).intersection(*intersection_list[1:])
            return random.choice(list(intersection))
        else:
            return None
    
    def generate_one_to_many(self,refrenced_tab,x):
        '''Filler function '''
        return 0
    def random_boolean(self, distribution_tab=None):
        # Check if a distribution tab is provided
        if distribution_tab:
            # Check if the distribution tab exists
            if distribution_tab not in self.tables:
                raise ValueError(f"Distribution tab '{distribution_tab}' not found.")

            # Get the index of the Boolean and Dist columns
            boolean_index = 0
            dist_index = 1

            # Get the Boolean and Dist columns
            booleans = [row[boolean_index] == "true" for _, row in self.tables[distribution_tab].iterrows()]
            dists = [float(row[dist_index]) for _,row in self.tables[distribution_tab].iterrows()]

            # Normalize the distribution percentages
            total_dist = sum(dists)
            normalized_dists = [dist / total_dist for dist in dists]

            # Generate a random boolean value based on the distribution
            random_boolean = random.choices(booleans, weights=normalized_dists)[0]
        else:
            # Generate a truly random boolean value
            random_boolean = random.choice([True, False])

        return random_boolean
    def parent(self,table_name,cc,j, parent_tab, column_name):
                # Check if the parent tab exists
        if parent_tab not in self.tables:
            raise ValueError(f"Parent tab '{parent_tab}' not found.")

        # Get the index of the specified column in the parent tab
        if column_name.strip() not in self.tables[parent_tab].iloc[0].to_list():
            raise ValueError(f"Column '{column_name}' not found in parent tab '{parent_tab}'.")
        df = pd.read_csv(f"{parent_tab}.csv")
        
        j=df[df.iloc[:,0]==j][column_name].to_list()[-1] # take the last created value of the column in the row of the parent tab
        

   
       
        return j
    def parentt(self,table_name,l,cc,j, parent_tab, column_name):
        '''
        Function to deal with nested parent calls
        '''
                # Check if the parent tab exists
        if parent_tab not in self.tables:
            raise ValueError(f"Parent tab '{parent_tab}' not found.")

        # Get the index of the specified column in the parent tab
        if column_name.strip() not in self.tables[parent_tab].iloc[0].to_list():
            raise ValueError(f"Column '{column_name}' not found in parent tab '{parent_tab}'.")
        df = pd.read_csv(f"{parent_tab}.csv")
        
       
        if f"{parent_tab}ID" in self.tables[table_name].iloc[0].to_list()[0] :

            r = df[df[f"{parent_tab}ID"]==j][column_name.strip()]

            return r.to_list()[0] 
        else :
            df = pd.read_csv(f"{str(self.tables[table_name].iloc[0].to_list()[1]).removesuffix('ID')}.csv")

            j=df[df.iloc[:,0]==j][f"{parent_tab}ID"].to_list()[0]
            return self.parentt(str(self.tables[table_name].iloc[0].to_list()[1]).removesuffix("ID"),l,cc,j,parent_tab,column_name)



        
    def calculate_years(self,l, columns , c ,date1, date2):
        date1 = date1.strip()
        date2 = date2.strip()
      
        if date1 in columns:
                i = columns.index(date1)
                date1 = l[i]
        if date2 in columns:
            i = columns.index(date2)
            date2 = l[i]

        # Resolve date references if any
        date1 = self.resolve_date_reference(date1)
        date2 = self.resolve_date_reference(date2)

        # Convert the date parameters to datetime objects
        date1 = datetime.strptime(str(date1), "%Y%m%d").date()
        date2 = datetime.strptime(str(date2), "%Y%m%d").date()

        # Calculate the absolute difference in years
        absolute_years = abs((date1 - date2).days) / 365.25

        return int(round(absolute_years, 0))
        
    def pull_from(self, ll,table_name,c,token_name, referenced_tab, condition_expr):
        # Check if the referenced tab exists
        referenced_tab = referenced_tab.strip()
        if referenced_tab not in self.tables:
            raise ValueError(f"Referenced tab '{referenced_tab}' not found.")

        # Extract column names from the condition expression
        column_names = [col.strip() for col in condition_expr.split('=')]
        column_namess = column_names[-1]
        cond =column_names[0].split(".")[-1].strip()
        i = self.tables[table_name].iloc[0].to_list().index(column_namess) # the index of the column
        v = ll[i] # the value of the column in the current row 

        # Check if the columns exist in the referenced tab
        for col_name in [cond]:
            if col_name not in self.tables[referenced_tab].iloc[0].to_list():
                raise ValueError(f"Column '{col_name}' not found in referenced tab '{referenced_tab}'.")
        # add the data as a token
        df = pd.read_csv(f"{referenced_tab}.csv")
        self.tokens[token_name] = {"data" : df[df[cond]==v]}
        self.tokens[token_name]["columns"] = df.columns

       

        return np.nan

    def random_select(self, token_name, source_tab_or_token):
        # Check if the source is a token or a tab
        if source_tab_or_token in self.tokens:
            source_data = self.tokens[source_tab_or_token]["data"]
            col = self.tokens[source_tab_or_token]["columns"]
        elif source_tab_or_token in self.tables:
            df = self.tables[source_tab_or_token]
            source_data = df
            col = df.columns
        else:
            raise ValueError(f"Source '{source_tab_or_token}' not found.")

        # Randomly select an entire row
        selected_row = random.choice([row for _ ,row in source_data.iterrows()])

        # Store the selected row and its columns as a token
        self.tokens[token_name]={"data" : selected_row}
        self.tokens[token_name]["columns"] = col

        return selected_row.to_list()

    def lookup_with_token(self, token_name, column_name):
        # Check if the token exists
        if token_name not in self.tokens:
            raise ValueError(f"Token '{token_name}' not found.")

        # Get the selected row and its columns from the token
        selected_row = self.tokens[token_name]["data"]
        source_columns = self.tokens[token_name]["columns"]

        # Check if the column exists in the selected row
        if column_name not in source_columns:
            raise ValueError(f"Column '{column_name}' not found in selected row.")

        # Return the value from the specified column in the selected row
        column_index = source_columns.to_list().index(column_name)
        return selected_row[column_index]

    def calculate_integer(self,table_name, ll,left_value, right_value, operator):
        # Convert values to integers to ensure they are numeric
        params = [left_value, right_value]
        for i,v in enumerate(params):
            # deal with non numeric entries
            try :
                v = int(v)
                params[i] = v 
               
            except :
                j = self.tables[table_name].iloc[0].to_list().index(v) 
                v = int(ll[j])
                params[i] = v 
        left_value,right_value = params

        # Perform the specified operation
        if operator in '"+"':
            result = left_value + right_value
        elif operator == '"-"':
            result = left_value - right_value
        elif operator == '"*"':
            result = left_value * right_value
        elif operator == '"/"':
            if right_value == 0:
                raise ValueError("Division by zero is not allowed.")
            result = left_value / right_value
        else:
            raise ValueError(f"Invalid operator: {operator}")

        # Truncate the result to an integer
        result = int(result)

        return result

    def calculate_decimal(self, left_value, right_value, operator, significant_digits):
        # Convert values to floats to handle decimals
        left_value = float(left_value)
        right_value = float(right_value)

        # Perform the specified operation
        if operator == "+":
            result = left_value + right_value
        elif operator == "-":
            result = left_value - right_value
        elif operator == "*":
            result = left_value * right_value
        elif operator == "/":
            if right_value == 0:
                raise ValueError("Division by zero is not allowed.")
            result = left_value / right_value
        else:
            raise ValueError(f"Invalid operator: {operator}")

        # Round the result to the specified number of decimal places
        result = round(result, significant_digits)

        return result
    def condition(self,table_name,ll,j,cc, condition_expr, true_value, false_value,gen=1):
        l = []
        if "=" in condition_expr:
            c1,c2 = condition_expr.split("=")
            #increment genearte currrent count for each row created
            if "GenerateCurrentCount" in c1 :
                
                    t = pd.read_csv(f"{table_name}.csv")
                    if len( list(t.iterrows())) >0:
                     
                        
                        for i , row in t.iterrows():
                            
                            if int(row.tolist()[1]) == int(j):
                                gen+=1
                            
                    c1 = f"{gen}"
                
        else: # if its already a boolean
            c1 = condition_expr
            c2="LALA"
        #deal with nested commands
        for param in [c1, true_value, false_value]:
            if param == "Sequential":
                l.append(self.generate_data("Sequential"))
            
            elif "(" in param :
                param = param.split("(",1)
                command_name = param[0]
                
                aras = param[-1].split(",",1)
                aras = [a.rstrip(')') for a in aras]
                
                l.append(self.generate_data(command_name,ll=ll,table_name=table_name,c= cc,parameters = aras,j=j))
            else:
                l.append(param)
        if c2 !="LALA": 
            l[0] = (l[0]==c2)
            condition_expr = l[0]
        # Evaluate the condition expression
        if isinstance(l[0], bool):
            condition_result = condition_expr == True
        elif isinstance(l[0], str) and l[0].upper() in ["TRUE", "FALSE"]:
            condition_result = condition_expr.upper() == "TRUE"
        else:
            operator, comparison_value = self.parse_condition_expression(condition_expr)

            # Evaluate the condition
            if self.evaluate_condition(operator, comparison_value):
                condition_result = True
            else:
                condition_result = False
        return l[1] if condition_result else l[2]

    def parse_condition_expression(self, condition_expr):
        # Split the condition expression into operator and comparison value
        operators = ["<", ">", "<=", ">=", "<>", "="]
        for op in operators:
            if op in condition_expr:
                operator, comparison_value = condition_expr.split(op, 1)
                operator = op
                comparison_value = comparison_value.strip()
                return operator, comparison_value

        # If no valid operator is found, default to equality
        return "=", condition_expr.strip()
    def concat(self,*params):
        l = []
        # resolve nested commands
        for param in params:
            if param == "Sequential":
                l.append(self.generate_data("Sequential"))
            elif "(" in param :
                param = param.split("(")
                command_name = param[0]
                if "," in command_name :
                    cc = command_name.split(",")
                    command_name = cc[1]
                    l.append(cc[0])

                params = [p.rstrip(")") for p in param]
                aras = params[1].split(",")
                l.append(self.generate_data(command_name,parameters = aras))
            else:
                l.append(param)
        
        f=''.join([str(x) for x in l]) #join without space 
        return f.replace("'",'') # eliminate the extra quotes
        

    def evaluate_condition(self, operator, comparison_value):
        # Evaluate the condition based on the operator
        if operator == "=":
            return self.generate_current_count() == int(comparison_value)
        elif operator == "<":
            return self.generate_current_count() < int(comparison_value)
        elif operator == ">":
            return self.generate_current_count() > int(comparison_value)
        elif operator == "<=":
            return self.generate_current_count() <= int(comparison_value)
        elif operator == ">=":
            return self.generate_current_count() >= int(comparison_value)
        elif operator == "<>":
            return self.generate_current_count() != int(comparison_value)

    def generate_current_count(self):
        # Placeholder method to generate the current count
        return self.counter
    def generate_data(self,command_name,table_name=None,ll=None,c=None,parameters=None,j=None):
        '''
        This generate data is used for nested commands
        '''
        if command_name == "Sequential" :
            return self.generate_sequential()
        elif command_name == "RandomLookup":
            return self.random_lookup(*parameters)
       
        elif command_name == "RandomInteger" :
            return self.random_integer(*parameters)
        
    
        elif command_name == "GenerateOneToMany" :
            return self.generate_one_to_many(*parameters)
        elif command_name == "LookupWithInteger":
            return self.lookup_with_integer(*parameters)
        elif command_name == "RandomDate" :
            return self.random_date(ll,table_name,j,c,*parameters)
        elif command_name == "CalculateInteger" :
           return self.calculate_integer(*parameters)
        elif command_name == "CalculateDecimal" :
            return self.calculate_decimal(*parameters)
        elif command_name == "PullFrom":
            return self.pull_from(*parameters)
        elif command_name == "RandomSelect" :
            return self.random_select(*parameters)
        elif command_name == "Condition" :
       
            return self.condition(table_name,ll,j,c,*parameters)
        elif command_name == "LookupWithToken":
            return self.lookup_with_token(*parameters)
        elif command_name == "Parent":
            return self.parentt(table_name,ll,c,j,*parameters)
        elif command_name == "RandomIntersection":
            return self.random_intersection(ll,c,table_name,*parameters)
        elif command_name == "CalculateYears":
            return self.calculate_years(*parameters)
        elif command_name == "RandomBoolean":
            return self.random_boolean(*parameters)
        elif command_name == "Concat":
            return self.concat(*parameters)
        else:   
            return True






In [174]:
import csv
import re
import os
allowed_commands = [
            "Sequential",
            "RandomLookup",
            "RandomInteger",
            "RandomDate",
            "RandomIntersection",
            "RandomBoolean",
            "GenerateOneToMany",
            "LookupWithInteger",
            "Parent",
            "Condition",
            "CalculateYears",
            "PullFrom",
            "RandomSelect",
            "LookupWithToken",
            "CalculateInteger",
            "CalculateDecimal",
            "Random",
            "Concat"
        ]
class ExcelParser:
    def __init__(self, excel_path):
        self.excel_path = excel_path
        self.generator = SyntheticDataGenerator()
        self.doesnt_have_commands = {}
        self.does_have_commands = {}
        self.reference_tabs = {}
    

    def parse_excel(self):
        # Parse each sheet in the Excel file
        sheets = pd.read_excel(self.excel_path, sheet_name=None,parse_dates=True)
        for sheet_name, sheet_data in sheets.items():
            self.generator.add_table(sheet_name, sheet_data)
        for sheet_name, sheet_data in sheets.items():
            self.parse_sheet(sheet_name, sheet_data)
    def process_master_tab(self):
        master_data = self.generator.tables['Master']

        for _, row in master_data.iterrows():
            instruction = row['Instructions']
            table_name = row['Table']
            count = int(row['Count'])

           

            if instruction == 'Generate':
                self.generate_data_for_table(table_name, count)
        # eliminate unwanted columns and duplicates
        for sheet_name in self.does_have_commands.keys():
            final = pd.read_csv(f"{sheet_name}.csv")
            for col in final.columns:
                if col in ["CHILDRECORD","SUBSET","ROW"] or re.match(r'CHILDRECORD\.(\d+)',col):
                    final.drop(col,axis=1,inplace=True)
            final.drop_duplicates(inplace=True)
            final.to_csv(f"{sheet_name}.csv",index = False)
    def generate_data_for_table(self, table_name, count, j=None, t=None, tab_j=None):
        '''
        j :  row id 
        t : table name that contains generate one to many
        tab_j : dictionnary to hold generated rows for already created tables
        '''
        # Generate the name of the CSV file based on the table_name
        csv_file = f"{table_name}.csv"
        
        # Check if the CSV file already exists
        file_exists = os.path.isfile(csv_file)
        
        # Retrieve the table object based on the table_name
        table = self.generator.tables[table_name]

        # If the CSV file doesn't exist, create it and write column names
        if not file_exists:
            with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
                column_names = list(table.iloc[0].to_list())
                writer = csv.writer(file)
                writer.writerow(column_names)
        
        # If tab_j is not provided, initialize it as an empty dictionary
        if tab_j is None:
            tab_j = {}

        # Check if the table_name exists in self.does_have_commands
        if not table_name in self.does_have_commands:
            return f"{table_name} does not exist or does not have commands with the right syntax"
        else:
            # If the table name exists and has the right syntax, proceed with generating data
            table = self.generator.tables[table_name]
            
            # Get a list of instructions (assumed to be column names)
            instructions = list(table.columns)
            
            # Get column names
            columns = table.iloc[0].to_list()
            ll = []  # Initialize an empty list for storing generated data
            
            # Loop to generate data 'count' times
            for c in range(count):
                l = []  # Initialize an empty list to store data for one row
                
                # Loop through each instruction (column)
                for inst in instructions:
                    if inst[-2] == ".":
                        inst = inst[:-2]
                    
                    # Extract parameters and command name from the instruction
                    params, command_name = self.extract_params(inst)
                    
                    if len(params) > 0:
                        if params[-1] == '':
                            params.pop(-1)
                    params = [p.strip() for p in params]
                    
                    # If no parameters, generate data for the current instruction
                    if len(params) == 0:
                        l.append(self.generate_data(table_name, command_name))
                    
                    # Handle a specific case where the command name contains "GenerateOne"
                    elif 'GenerateOne' in command_name:
                        try:
                            # Find the index of the column then the id of the row
                            i = self.generator.tables[table_name].iloc[0].to_list().index(params[0])
                            tab_j[params[1]] = l[0]
                            t = table_name
                            l.append(0)
                            jj = l[i]
                        except:
                            # deal with the case where the param is a number 
                            tab_j[params[1]] = l[0]
                            t = table_name
                            l.append(0)
                            jj = int(params[0])
                        
                        # Append the data to the CSV file
                        csv_file = f"{table_name}.csv"
                        file_exists = os.path.isfile(csv_file)
                        
                        if not file_exists:
                            column_names = list(table.iloc[0].to_list())
                            
                            with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
                                writer = csv.writer(file)
                                writer.writerow(column_names)
                        
                        with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
                            writer = csv.writer(file)
                            writer.writerow(l)
                        
                        # Recursively call the function to generate data for the next table
                        self.generate_data_for_table(params[1], jj, j=tab_j[params[1]], t=t)
                    
                    else:
                        # Generate data for the current instruction with parameters
                        l.append(self.generate_data(table_name, command_name, columns, l, c, params, j))
                
                # Append the generated row data to the list
                ll.append(l)
                
                # Check if the CSV file exists (again)
                csv_file = f"{table_name}.csv"
                file_exists = os.path.isfile(csv_file)
                
                if not file_exists:
                    # If the CSV file doesn't exist, create it and write column names
                    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
                        column_names = list(table.iloc[0].to_list())
                        writer = csv.writer(file)
                        writer.writerow(column_names)
                    
               
                
                # Append the generated row data to the CSV file
                with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    # Write the header (if necessary)
                    # Write the data rows
                    writer.writerow(l)


    def parse_sheet(self, sheet_name, sheet_data):
            '''we change b to false if we detect an invalid syntax'''
            if any("(" in str(column) for column in sheet_data.columns):
            
                pot_instructions = list (sheet_data.columns)
                b = True
                
                for inst in pot_instructions:
                    if inst[-2] ==".":
                        inst = inst[:-2]
                    if isinstance(inst,str):
                        if self.validate_instruction_syntax(inst,sheet_name) == False :
                            b=False
                        

                    else : 
                        b=False
                if b :
                    self.does_have_commands[sheet_name]= sheet_data
                    print(f"{sheet_name} has commands")

                else : 
                    self.doesnt_have_commands[sheet_name] = sheet_data
                    print(f"{sheet_name} has syntax problems")
            else :
                self.reference_tabs[sheet_name] = sheet_data
    def extract_params(self,instruction):


        # Split the instruction to identify the command and its parameters
        command_parts = instruction.split("(")
        command_name = command_parts[0].strip()

        # Extract parameters
        if len(command_parts) == 2:
            parameters = command_parts[1].rstrip(")").split(",")
            num_parameters = len(parameters)
        elif len(command_parts)>2:# nested commands detected
            for i , x in enumerate(command_parts):
                #conditions to identify the nested commands
                if "," in x and ")" not in x :
                    y=x.split(",")
                    c = command_parts[i+1]
                    for p in range(len(y)-1):
                        command_parts[i+p] = y[p]
                    try:
                        command_parts[i+len(y)-1] = f'{y[-1]}({c.rstrip(")")})'
                    except :
                        command_parts.append(f'{y[-1]}({c.rstrip(")")})')
                elif "," in x and "=" in x and command_name == "Condition" :
                    if len(command_parts)==3:
                        y=x.split("=")
                        yy = y[1].split(",")
                        command_parts[-1] = command_parts[-1].strip(')')
                        command_parts[i-1] = f"{command_parts[i-1]}({y[0]}={yy[0]}"
                        command_parts[i] = f"{yy[-1]}({command_parts[-1]})"

                        command_parts.pop(-1)
                    
                    
            for i,x in enumerate(command_parts): 

                if ")" in x and not "(" in x:
                    y=x.split(")")
                    y = [l.lstrip(",") for l in y]
                    command_parts[i-1] = f"{command_parts[i-1]}({y[0]})"
                    command_parts[i] = y[1]
        
                
            for i in allowed_commands :
                if i in command_parts[1:] : 
                    ii = command_parts.index(i)
                
                    command_parts[ii] = f"{i}({command_parts[ii+1].strip()},{command_parts[ii+2].strip()})"
                    
                    xx = command_parts.pop(ii+1)
                    yy = command_parts.pop(ii+1)
            for i,x in enumerate(command_parts):
                if x.startswith('"') and "," in x:
                    command_parts[i] = x.lstrip('"').split('"')[-1]
                    command_parts[i] = command_parts[i].lstrip(',')
                elif "," in x and "=" in x and command_parts[1].startswith('Parent') :
                    y=x.split(",",1)
                    command_parts[-1] = command_parts[-1].strip(')')
                    command_parts[i-1] = f"{command_parts[i-1]}{y[0]}"
                    command_parts[i] = y[-1]
            command_parts = [l.lstrip(",") for l in command_parts]
            c = command_parts[2].rstrip(")").split(")")
            cc= c[-1].lstrip(",").split(",")
            num_parameters = len(cc)+1
           
            parameters  = command_parts[1:]
        else:
            parameters=[]
        return parameters , command_name
    def validate_instruction_syntax(self, instruction,sheet_name):
       
        allowed_commands = [
            "Sequential",
            "RandomLookup",
            "RandomInteger",
            "RandomDate",
            "RandomIntersection",
            "RandomBoolean",
            "GenerateOneToMany",
            "LookupWithInteger",
            "Parent",
            "Condition",
            "CalculateYears",
            "PullFrom",
            "RandomSelect",
            "LookupWithToken",
            "CalculateInteger",
            "CalculateDecimal",
            "Concat"
        ]
        
        # Split the instruction to identify the command and its parameters
        command_parts = instruction.split("(")
        command_name = command_parts[0].strip()
        
        if "+" in command_parts:#not useful anymore with concat 
            return True
        
        if "=" not in command_name and command_name not in allowed_commands:
            print(f"Invalid command {command_name} on {sheet_name} for {instruction}")
            return False
        parameters = self.extract_params(instruction)[0]
        num_parameters = len(parameters)
        """    
         # Validate each parameter
        for param in parameters:
            if not validate_parameter(param):
                return False
        """
        # Validate the number of parameters based on the command
        parameters = [p.strip() for p in parameters]
        if command_name == "Sequential" :
            return self.validate_sequential(parameters,sheet_name)
        elif command_name == "RandomLookup":
            return self.validate_random_lookup(parameters,sheet_name)
       
        elif command_name == "RandomInteger" :
            return self.validate_random_integer(parameters,sheet_name)
        
    
        elif command_name == "GenerateOneToMany" :
            return self.validate_generate_one_to_many(parameters,sheet_name)
        elif command_name == "LookupWithInteger" and num_parameters != 2:
            return self.validate_lookup_with_integer(parameters,sheet_name)
        elif command_name == "RandomDate" and num_parameters != 2:
            return self.validate_random_date(parameters,sheet_name)
        elif command_name == "CalculateInteger" and num_parameters != 3:
           return self.validate_calculate_integer(parameters)
        elif command_name == "CalculateDecimal" and num_parameters != 4:
            return self.validate_calculate_decimal(parameters)
        elif command_name == "PullFrom":
            return self.validate_pull_from(parameters,sheet_name)
        elif command_name == "RandomSelect" :
            return self.validate_random_select(parameters,sheet_name)
        elif command_name == "Condition" :
            return self.validate_condition(parameters,sheet_name)
        elif command_name == "LookupWithToken":
            return self.validate_lookup_with_token(parameters,sheet_name)
        elif command_name == "Parent":
            return self.validate_parent(parameters,sheet_name)
        elif command_name == "RandomIntersection":
            return self.validate_random_intersection(parameters,sheet_name)
        elif command_name == "CalculateYears":
            return self.validate_calculate_years(parameters,sheet_name)
        elif command_name == "RandomBoolean":
            return self.validate_random_boolean(parameters,sheet_name)
        elif command_name == "Concat":
            return self.validate_concat(command_parts,sheet_name)
        else:   
            return True
    def validate_concat(self, parameters,sheet_name):
       
        if not parameters:
            print("No parameters provided for Concat command")
            return False

        # Validate each parameter, supporting embedded commands or strings
        for param in parameters:
           
            if not self.validate_embedded_command(param,sheet_name):
                return False

        return True
        
    def validate_sequential(self, parameters,sheet_name):
        # Validate Sequential command parameters
        if len(parameters) != 0:
            print(f"Invalid number of parameters for Sequential command in {sheet_name}")
            return False
        else :
            return True
    def validate_random_lookup(self, parameters,sheet_name):
        # Validate RandomLookup command parameters
        if len(parameters) != 1:
            print(f"Invalid number of parameters {parameters} for RandomLookup command in {sheet_name}")
            return False
        if len(parameters) == 1 and (parameters[0] not in list(self.generator.tables.keys())):
            print(f"Tab {parameters[0]} not found for Random Lookup on {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        for param in parameters:
            if ("(" in param) and not self.validate_embedded_command(param,sheet_name, expected_return_type=int):
                return False

        return True
    def validate_random_integer(self, parameters,sheet_name):
        # Validate RandomInteger command parameters
        if len(parameters) not in [2, 3]:
            print(f"Invalid number of parameters for RandomInteger command in {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        
        if not parameters[0].strip().isdigit() or not parameters[1].strip().isdigit():
                print(f"Parameter:{parameters}")
                print(parameters[0].isdigit(), parameters[1].isdigit())
                print(f"Invalid parameter type {parameters[1]} or {parameters[0]} for RandomInteger command in {sheet_name}")
                return False
        if len(parameters) == 3 and parameters[2] not in  list(self.generator.tables.keys()):
            print( f"Distribution tab  {parameters[2]} not found for RandomInteger in {sheet_name}")

            return False
        for param in parameters:
            if not self.validate_embedded_command(param,sheet_name, expected_return_type=int):
                return False
        
        return True
    def validate_lookup_with_integer(self, parameters,sheet_name):
        # Validate LookupWithInteger command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for LookupWithInteger command in {sheet_name}")
            return False

        # Validate the first parameter, which is an integer or an embedded command
        if not parameters[0].isdigit() and not self.validate_embedded_command(parameters[0], sheet_name,expected_return_type=int):
            print(f"Invalid parameter type {parameters[0]}  for LookupWithInteger command in {sheet_name}")
            return False

        # Validate the second parameter, which is the referenced tab
        if parameters[1] not in list(self.generator.tables.keys()):
            print(f"Referenced tab '{parameters[1]}' not found for LookupWithInteger command")
            return False

        # Validate the structure of the referenced tab
        referenced_tab_columns = list(self.generator[parameters[1]].columns)
        if len(referenced_tab_columns) != 2 or "Value" not in referenced_tab_columns:
            print(f"Invalid structure for the referenced tab {parameters[1]} in LookupWithInteger command {sheet_name}")
            return False

        return True
    def validate_random_date(self, parameters, sheet_name):
        # Validate RandomDate command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for RandomDate command in {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        for param in parameters:
            if not self.validate_embedded_command(param,sheet_name, expected_return_type=str):
                return False

        return True
    def validate_random_intersection(self, parameters,sheet_name):
        # Validate RandomIntersection command parameters
        if len(parameters) % 2 != 0 or len(parameters) < 4:
            print(f"Invalid number of parameters for RandomIntersection command in {sheet_name}")
            return False

        # Validate each pair of parameters, supporting embedded commands
        for i in range(0, len(parameters), 2):
            tab_param = parameters[i]
            column_param = parameters[i + 1]
            if tab_param not in list(self.generator.tables.keys()):
                print(f"Tab '{tab_param}' does not exist for RandomIntersection command in {sheet_name}")
                return False
            # Validate tab parameter
            if not self.validate_embedded_command(tab_param, sheet_name,expected_return_type=list):
                return False

            # Validate column parameter
            if not self.validate_embedded_command(column_param,sheet_name, expected_return_type=str):
                return False

           
    
        return True
    def validate_random_boolean(self, parameters,sheet_name):
        # Validate RandomBoolean command parameters
        if len(parameters) not in [0, 1]:
            print(f"Invalid number of parameters for RandomBoolean command in {sheet_name}")
            return False

        # If a distribution tab is provided, validate it
        if len(parameters) == 1:
            distribution_tab_param = parameters[0]
            if distribution_tab_param not in list(self.generator.tables.keys()):
                print(f"Distribution tab '{distribution_tab_param}' does not exist for Random in {sheet_name}")
                return False
                      
            # Validate distribution tab parameter
            if not self.validate_embedded_command(distribution_tab_param,sheet_name, expected_return_type=list):
                return False

            # Validate the existence of the specified columns in the distribution tab
            distribution_tab_data = self.generator.tables[distribution_tab_param]
            if "Dist" not in distribution_tab_data.columns:
                print(f"Validation failed for RandomBoolean command: Distribution tab '{distribution_tab_param}' must have 'Dist' columns")
                return False

        return True
    def validate_generate_one_to_many(self, parameters,sheet_name):
        # Validate GenerateOneToMany command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for GenerateOneToMany command in {sheet_name}")
            return False

        # Validate the first parameter (number of rows to create)
        num_rows_param = parameters[0]
        if  num_rows_param not in self.generator.tables[sheet_name].iloc[0].tolist() and not num_rows_param.isdigit() :
            print(f"Invalid parameter  {num_rows_param} for GenerateOneToMany command (first parameter must be an integer) in {sheet_name}")
            return False

        # Validate the second parameter (child tab)
        child_tab_param = parameters[1]
        if child_tab_param not in list(self.generator.tables.keys()):
            print(f"Invalid child tab '{child_tab_param}' in GenerateOneToMany command in {sheet_name}")
            return False

        

        return True
    def validate_parent(self, parameters,sheet_name):
        # Validate Parent command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for Parent command in {sheet_name}")
            return False

        # Validate the first parameter (parent tab)
        parent_tab_param = parameters[0]
        if parent_tab_param not in self.generator.tables.keys():
            print(f"Invalid parent tab '{parent_tab_param}' in Parent command in {sheet_name}")
            return False

        # Validate the second parameter (column reference)
        column_reference_param = parameters[1]
        if column_reference_param.strip() not in self.generator.tables[parent_tab_param].columns and (column_reference_param.strip() not in self.generator.tables[parent_tab_param].iloc[0].to_list()) and "=" not in column_reference_param :
            print(f"Invalid column reference '{column_reference_param}' in Parent command for tab '{parent_tab_param}'")
            return False

        return True
    def validate_condition(self, parameters,sheet_name):
        
        # Validate the second and third parameters based on the type of the first parameter
        for param in parameters:
            # If the condition expression is a boolean expression, validate the second and third parameters as appropriate
            if not self.validate_embedded_command(param,sheet_name) :
                return False
        

        return 
    def validate_calculate_years(self, parameters,sheet_name):
        # Validate CalculateYears command parameters
        if len(parameters) != 2:
            print("Invalid number of parameters for CalculateYears command")
            return False

        # Validate each parameter, supporting embedded commands
        for param in parameters:
            if not self.validate_embedded_command(param, sheet_name,expected_return_type=str):
                print("Invalid parameter type for CalculateYears command")
                return False

        return True
    def validate_pull_from(self, parameters,sheet_name):
        # Validate PullFrom command parameters
        if len(parameters) != 3:
            print(f"Invalid number of parameters for PullFrom command in {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        token_name, referenced_tab, condition_expr = parameters

        # Validate token_name
        if not token_name.isidentifier():
            print(f"Invalid token name for PullFrom command in {sheet_name}")
            return False
        referenced_tab = referenced_tab.strip()
        # Validate referenced_tab
        if referenced_tab.strip() not in self.generator.tables.keys():
            print(f"Referenced tab '{referenced_tab}' not found for PullFrom command in {sheet_name}")
            return False

        # Validate condition_expr
        condition_expr_split = condition_expr.split("=")
        if len(condition_expr_split) != 2:
            print(f"Invalid condition expression for PullFrom command in {sheet_name}")
            return False

        condition_column, condition_value = map(str.strip, condition_expr_split)
        condition_column = condition_column.split(".")[1]
        if condition_column not in self.generator.tables[referenced_tab].iloc[0].to_list():
            print(f"Column '{condition_column}' not found in referenced tab '{referenced_tab}' for PullFrom command in {sheet_name}")
            return False

        # Validate condition_value (allowing for embedded commands)
        if not self.validate_embedded_command(condition_value,sheet_name):
            return False

        return True
    def validate_random_select(self, parameters,sheet_name):
        # Validate RandomSelect command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for RandomSelect command in {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        token_name, source_tab_or_token = parameters

        return True

    
        # Validate source_tab_or_token : this needs to be handled after data generation from pullfrom
        
        if token_name not in self.generator.tables.keys() and token_name not in self.generator.tokens.keys():
            print(f"Source '{token_name}' not found for RandomSelect command")
            return False

    
    def validate_lookup_with_token(self, parameters,sheet_name):
        # Validate LookupWithToken command parameters
        if len(parameters) != 2:
            print(f"Invalid number of parameters for LookupWithToken command in {sheet_name}")
            return False

        # Validate each parameter, supporting embedded commands
        token_name, column_name = parameters
      
        # Validate token_name this should be done in generation
        """
        if not token_name.isidentifier() or not token_name in self.generator.tokens.keys():
            print("Invalid token name for LookupWithToken command")
            return False
        """
        # Validate column_name
        if not isinstance(column_name,str):
            print(f"Invalid column name {column_name} for LookupWithToken command in {sheet_name}")
            return False

        return True
    def validate_calculate_integer(self, parameters):
        # Validate CalculateInteger command parameters
        if len(parameters) != 3:
            print("Invalid number of parameters for CalculateInteger command")
            return False

        # Validate each parameter, supporting embedded commands
        left_value, right_value, operator = parameters

        # Validate left_value and right_value
        if not left_value.isnumeric() or not right_value.isnumeric():
            print("Invalid parameter type for CalculateInteger command")
            return False

        # Validate operator
        if operator not in ["+", "-", "*", "/"]:
            print("Invalid operator for CalculateInteger command")
            return False

        # Validate division by zero
        if operator == "/" and right_value == "0":
            print("Division by zero is not allowed for CalculateInteger command")
            return False

        return True

    def validate_calculate_decimal(self, parameters):
        # Validate CalculateDecimal command parameters
        if len(parameters) != 4:
            print("Invalid number of parameters for CalculateDecimal command")
            return False

        # Validate each parameter, supporting embedded commands
        left_value, right_value, operator, significant_digits = parameters

        # Validate left_value and right_value
        if not left_value.replace(".", "", 1).isdigit() or not right_value.replace(".", "", 1).isdigit():
            print("Invalid parameter type for CalculateDecimal command")
            return False

        # Validate operator
        if operator not in ["+", "-", "*", "/"]:
            print("Invalid operator for CalculateDecimal command")
            return False

        # Validate significant_digits
        if not significant_digits.isdigit():
            print("Invalid significant_digits for CalculateDecimal command")
            return False

        return True
    
    def validate_embedded_command(self, embedded_command,sheet_name ,expected_return_type=None):
        
        allowed_commands = [
            "Sequential",
            "RandomLookup",
            "RandomInteger",
            "RandomDate",
            "RandomIntersection",
            "RandomBoolean",
            "GenerateOneToMany",
            "LookupWithInteger",
            "Parent",
            "Condition",
            "CalculateYears",
            "PullFrom",
            "RandomSelect",
            "LookupWithToken",
            "CalculateInteger",
            "CalculateDecimal",
            "Concat"
        ]
      
        if "(" not in embedded_command or ")" not in embedded_command:
            return True

        command_name, command_params = self.extract_command_name_and_params(embedded_command)

        # Validate if the command exists
        if "=" not in command_name and command_name not in allowed_commands:
            #print(f"Command '{command_name}' for {embedded_command} not found.")
            return True 

        # Validate the return type
        if expected_return_type is not None:
            if not self.validate_return_type(command_name, expected_return_type):
             return False
        
        # Validate the parameters recursively
        for param in command_params:
                if not self.validate_embedded_command(param, expected_return_type):
                    return False

        return self.validate_instruction_syntax(embedded_command,sheet_name)

    def extract_command_name_and_params(self, embedded_command):
      
        command_parts = embedded_command.replace("(", " ").replace(")", " ").split()
        command_name = command_parts[0]
        command_params = command_parts[1:]

        return command_name, command_params

    def validate_return_type(self, command_name, expected_return_type):
       
        return_type = self.get_command_return_type(command_name)

        if return_type is None:
            print(f"Return type for command '{command_name}' not defined.")
            return False

        return return_type == expected_return_type

    def get_command_return_type(self, command_name):
      
        command_return_types = {
                   
            "Sequential" : int ,
            "RandomLookup" : str,
            "RandomInteger" : int ,
            "RandomDate" : str ,
            "RandomIntersection" : str or int ,
            "RandomBoolean": bool,
            
            "LookupWithInteger" : str,
            "Parent" : int,
            
            "CalculateYears" : int,
            
            "RandomSelect" : str or int ,
      
            "CalculateInteger" : int,
            "CalculateDecimal" : float,
        
        
           
        }

        return command_return_types.get(command_name)
    def generate_data(self,table_name ,command_name,columns = None,ll=None,c=None,parameters=None,j=None,gen=1):
        if command_name == "Sequential" :
            return self.generator.generate_sequential(table_name)
        elif command_name == "RandomLookup":
            return self.generator.random_lookup(*parameters)
       
        elif command_name == "RandomInteger" :
            return self.generator.random_integer(*parameters)
        
    
        elif command_name == "GenerateOneToMany" :
            return self.generator.generate_one_to_many(*parameters)
        elif command_name == "LookupWithInteger":
            return self.generator.lookup_with_integer(table_name,j,*parameters,gen=gen)
        elif command_name == "RandomDate" :
            return self.generator.random_date(ll,table_name,j,c,*parameters)
        elif command_name == "CalculateInteger" :
           return self.generator.calculate_integer(table_name,ll,*parameters)
        elif command_name == "CalculateDecimal" :
            return self.generator.calculate_decimal(*parameters)
        elif command_name == "PullFrom":
            return self.generator.pull_from(ll,table_name,c,*parameters)
        elif command_name == "RandomSelect" :
            return self.generator.random_select(*parameters)
        elif command_name == "Condition" :
            try:
                return self.generator.condition(table_name,ll,j,c,*parameters,gen=gen)
            except:
                p = parameters[-1].split(",")
                parameters[-1] = p[0]
                parameters.append(p[1])
                return self.generator.condition(table_name,ll,j,c,*parameters,gen=gen)


        elif command_name == "LookupWithToken":
            return self.generator.lookup_with_token(*parameters)
        elif command_name == "Parent":
            return self.generator.parent(table_name,c,j,*parameters)
        elif command_name == "RandomIntersection":
            return self.generator.random_intersection(ll,c,table_name,*parameters)
        elif command_name == "CalculateYears":
            return self.generator.calculate_years(ll,columns,c,*parameters)
        elif command_name == "RandomBoolean":
            return self.generator.random_boolean(*parameters)
        elif command_name == "Concat":
            return self.generator.concat(*parameters)
        else:   
            return True

    def get_generated_data(self):
        # Get the generated data from the generator
        generated_data = self.generator.tables
        return generated_data

# Example usage
parser = ExcelParser(excel_path)
parser.parse_excel()
generated_data = parser.generate_data("Concat",["Sequential"," ","RandomLookup(LastName)","Street"])



NIINs has commands
Contracts has commands
SORs has commands
SORProduction has commands


In [175]:
parser.process_master_tab()

('RandomLookup(Alpha)', 'RandomLookup(Alpha)', "' ',RandomInteger(100000000000000,999999999999999)")
['H', 'F', "' '", 429593284951171]
('RandomInteger(1000,9999)', 'RandomLookup(Alpha)', 'RandomLookup(Alpha)')
[7925, 'E', 'D']
('RandomLookup(Alpha)', 'RandomLookup(Alpha)', "' ',RandomInteger(100000000000000,999999999999999)")
['C', 'E', "' '", 250625599475089]
('RandomInteger(1000,9999)', 'RandomLookup(Alpha)', 'RandomLookup(Alpha)')
[1398, 'B', 'F']
('RandomLookup(Alpha)', 'RandomLookup(Alpha)', "' ',RandomInteger(100000000000000,999999999999999)")
['A', 'E', "' '", 531741653138774]
('RandomInteger(1000,9999)', 'RandomLookup(Alpha)', 'RandomLookup(Alpha)')
[4709, 'B', 'A']
('RandomLookup(Alpha)', 'RandomLookup(Alpha)', 'RandomInteger(100000000000000,999999999999999)')
['C', 'G', 427728729085345]
('RandomLookup(Alpha)', 'RandomLookup(Alpha)', 'RandomInteger(100000000000000,999999999999999)')
['B', 'H', 288746413166988]
('RandomLookup(Alpha)', 'RandomLookup(Alpha)', 'RandomInteger(1000

In [180]:
parser.reference_tabs.keys()


dict_keys(['Master', 'ItemCatCodes', 'SORList', 'PlannerList', 'Alpha', 'ProdMonth'])